In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, log_loss, roc_auc_score

# Logistic regression (Classification of general recidivism)

## Load dataset

In [122]:
df = pd.read_csv("../../data-cleaned/compas-scores-two-years-clean.csv")

ProPublicas reasoning for filter `raw_data`:
>However not all of the rows are useable for the first round of analysis.
>There are a number of reasons remove rows because of missing data:
>* If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense.
>* We coded the recidivist flag -- `is_recid` -- to be -1 if we could not find a compas case at all.
>* In a similar vein, ordinary traffic offenses -- those with a `c_charge_degree` of 'O' -- will not result in Jail time are removed (only two of them).
>* We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.

## Filter dataset

In [13]:
df = df[
        (df["days_b_screening_arrest"]<=30) 
        & (df["days_b_screening_arrest"]>=-30) 
        & (df["is_recid"]!=-1) 
        & (df["c_charge_degree"]!="O") 
      ].dropna(subset=['score_text'])

In [109]:
# Select columns of interest and store as new df
columns = ["id", "age", "age_cat", "sex", "race",  #demographics
           "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", #criminal history
           "c_charge_degree", "c_charge_desc", #current charges
           "score_text", "decile_score", #compas results for current charges
           "r_charge_degree", "r_charge_desc", #commited crimes (general) within 2 years after COMPAS scoring 
           "two_year_recid"] #truth

df_clean = df[columns]

## Check values and dtypes of columns

In [120]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5304 entries, 0 to 6215
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               5304 non-null   int64 
 1   age              5304 non-null   int64 
 2   age_cat          5304 non-null   object
 3   sex              5304 non-null   object
 4   race             5304 non-null   object
 5   juv_fel_count    5304 non-null   int64 
 6   juv_misd_count   5304 non-null   int64 
 7   juv_other_count  5304 non-null   int64 
 8   priors_count     5304 non-null   int64 
 9   c_charge_degree  5304 non-null   object
 10  c_charge_desc    5299 non-null   object
 11  score_text       5304 non-null   object
 12  decile_score     5304 non-null   int64 
 13  r_charge_degree  2122 non-null   object
 14  r_charge_desc    2095 non-null   object
 15  two_year_recid   5304 non-null   int64 
dtypes: int64(8), object(8)
memory usage: 833.5+ KB


In [93]:
# Check c_charge_desc for "arrest case no charge"

# df_clean = df_clean.dropna()
# df_clean[df_clean["c_charge_desc"].str.contains("arrest case no charge")]

In [113]:
# There are 358 unique charges in the dataset in total (wo NaN)
print(len(df_clean.c_charge_desc.unique()))

# There are 265 unqiue charges for felonies (23 of them also exist in misdemeanor) (wo NaN)
print(len(df_clean[df_clean["c_charge_degree"]=="F"][["c_charge_degree", "c_charge_desc"]].c_charge_desc.unique()))

# There are 125 unqiue charges for misdemeanors  (wo NaN)
print(len(df_clean[df_clean["c_charge_degree"]=="M"][["c_charge_degree", "c_charge_desc"]].c_charge_desc.unique()))

359
266
126


## Select relevant features

In [126]:
#Select features for Logistic regression model age or age_cat, exclude/ include race
columns_log = ["age", "age_cat", "sex", "race",  #demographics
               "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", #criminal history
               "c_charge_degree", "c_charge_desc", #current charges 
               "two_year_recid"] #truth

df_log = df_clean[columns_log]

In [127]:
df_log

,age,age_cat,sex,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,c_charge_desc,two_year_recid
0,69,Greater than 45,Male,Other,0,0,0,0,F,Aggravated Assault w/Firearm,0
1,34,25 - 45,Male,African-American,0,0,0,0,F,Felony Battery w/Prior Convict,1
2,24,Less than 25,Male,African-American,0,0,1,4,F,Possession of Cocaine,1
5,44,25 - 45,Male,Other,0,0,0,0,M,Battery,0
6,41,25 - 45,Male,Caucasian,0,0,0,14,F,Possession Burglary Tools,1
...,...,...,...,...,...,...,...,...,...,...,...
6211,20,Less than 25,Male,African-American,0,0,0,0,F,Possession of Cocaine,0
6212,23,Less than 25,Male,African-American,0,0,0,0,F,Deliver Cannabis,0
6213,23,Less than 25,Male,African-American,0,0,0,0,F,Leaving the Scene of Accident,0
6214,57,Greater than 45,Male,Other,0,0,0,0,F,Aggravated Battery / Pregnant,0


## Create pipeline

## Baseline model